In [3]:
import pandas as pd
from yahooquery import Ticker

# Liste des symboles boursiers pour les entreprises
companies = ['XOM', 'AAPL', 'MSFT', 'GOOGL', 'TSLA', 'META']

# Télécharger les données financières pour chaque entreprise et les stocker dans une liste
data_frames = []
for company in companies:
    ticker = Ticker(company)
    income_statement_df = ticker.income_statement('q')
    # Ajouter une colonne pour le symbole de l'entreprise
    income_statement_df['Symbole'] = company
    data_frames.append(income_statement_df)

# Combiner les DataFrames en un seul DataFrame
combined_df = pd.concat(data_frames)

# Assurer que la colonne 'asOfDate' est au format datetime
combined_df['asOfDate'] = pd.to_datetime(combined_df['asOfDate'])

# Filtrer les données pour n'inclure que les années antérieures à 2023
sorted_by_year_df = combined_df[combined_df['asOfDate'].dt.year < 2023]

# Tri par année (utilisation de la fonction 'resample' pour trier par année)
sorted_by_year_df = sorted_by_year_df.set_index('asOfDate').groupby('Symbole').resample('Y').last()

# Sélectionner uniquement les colonnes spécifiées
columns_to_keep = ['CostOfRevenue', 'BasicEPS', 'EBITDA', 'GrossProfit', 'NetIncome', 'TotalRevenue', 'BasicAverageShares']
sorted_by_year_df = sorted_by_year_df[columns_to_keep]

# Calculer les ratios financiers pour chaque action
sorted_by_year_df['GrossProfitMargin'] = (sorted_by_year_df['GrossProfit'] / sorted_by_year_df['TotalRevenue']) * 100
sorted_by_year_df['NetProfitMargin'] = (sorted_by_year_df['NetIncome'] / sorted_by_year_df['TotalRevenue']) * 100
sorted_by_year_df['EBITDAMargin'] = (sorted_by_year_df['EBITDA'] / sorted_by_year_df['TotalRevenue']) * 100

# Sélectionner uniquement les colonnes avec les ratios calculés
columns_with_ratios = ['CostOfRevenue', 'BasicEPS', 'EBITDA', 'GrossProfit', 'NetIncome', 'TotalRevenue', 'BasicAverageShares',
                       'GrossProfitMargin', 'NetProfitMargin', 'EBITDAMargin']
sorted_by_year_df = sorted_by_year_df[columns_with_ratios]

# Réinitialiser l'index pour afficher les données sous forme de DataFrame
sorted_by_year_df.reset_index(inplace=True)

# Afficher les données sous forme de DataFrame
print("Données triées par année avec les ratios financiers calculés :\n", sorted_by_year_df)

print("Données triées par année (colonnes sélectionnées) :\n", sorted_by_year_df)


Données triées par année avec les ratios financiers calculés :
   Symbole   asOfDate  CostOfRevenue  BasicEPS  EBITDA   GrossProfit  \
0    AAPL 2022-12-31   6.682200e+10      1.89     NaN  5.033200e+10   
1   GOOGL 2022-12-31   3.534200e+10      1.06     NaN  4.070600e+10   
2    META 2022-12-31   8.336000e+09      1.76     NaN  2.382900e+10   
3    MSFT 2022-12-31   1.748800e+10      2.20     NaN  3.525900e+10   
4    TSLA 2022-12-31   1.854100e+10      1.18     NaN  5.777000e+09   
5     XOM 2022-12-31   6.619000e+10      3.09     NaN  2.697400e+10   

      NetIncome  TotalRevenue  BasicAverageShares  GrossProfitMargin  \
0  2.999800e+10  1.171540e+11        1.589272e+10          42.962255   
1  1.362400e+10  7.604800e+10        1.289800e+10          53.526720   
2  4.652000e+09  3.216500e+10        2.639000e+09          74.083631   
3  1.642500e+10  5.274700e+10        7.451000e+09          66.845508   
4  3.687000e+09  2.431800e+10        3.160000e+09          23.756065   
5  1.2